In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
date_columns = [
                'Account_Created_Date', 'Opportunity_Created_Date',
                'Quote_Expiry_Date', 'Last_Modified_Date',
                'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                ]

categorical_columns = [
        'Region', 'Territory', 'Bureaucratic_Code',
        'Source', 'Billing_Country', 'Account_Name',
        'Opportunity_Name', 'Account_Owner', 'Opportunity_Owner',
        'Account_Type', 'Opportunity_Type', 'Quote_Type',
        'Delivery_Terms', 'Brand', 'Product_Type',
        'Size', 'Product_Category_B', 'Currency',
        'Last_Modified_By', 'Product_Family', 'Product_Name',
        'ASP_Currency', 'ASP_(converted)_Currency', 'Delivery_Quarter',
        'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Stage',
        'Prod_Category_A'
    ]

# Columnnas excluidas porque: - tienen igual valor en todos sus registros
#                             - no es valido usarlas
#                             - ya se extrajo información relevante de ellas
#                             - no aporta información relevante
excluded = ['Actual_Delivery_Date', 'Last_Activity',
        'Submitted_for_Approval','Prod_Category_A']

In [3]:
def open_set(dataset):
    
    column_types = { col:'category' for col in categorical_columns }
    
    # read_csv
    df = pd.read_csv(dataset, parse_dates=date_columns, dtype=column_types,
                     index_col='ID', na_values=['Other', 'NaT', 'None'],
                     usecols=lambda x: x not in excluded)

    df['Month'] = pd.to_datetime(df['Month'], format='%Y - %m')
    
    # Agruping regions 
    df.loc[((df.Region == "EMEA")&(df.Territory.str.contains("America"))), "Region"] = "Americas"
    df.rename(columns={"Source ":"Source"},inplace=True)
    df["Source"]= df["Source"].astype("category")

    return df

# Pre-procesamiento de los datos

El objetivo de esta etapa, es recibir los datos "crudos" y realizar procedimientos necesarios para filtrar features de poco valor y crear otros features que revelen información de importancia, para que los modelos de machine learning que luego los utilizarán en una etapa posterior, puedan ralizar un predicción mas precisa.

In [4]:
df1 = open_set("rawdata/Train_TP2_Datos_2020-2C.csv")
df2 = open_set("rawdata/Test_TP2_Datos_2020-2C.csv")

In [5]:
print(f"The train set has {df1.shape[0]} elements and {df1.shape[1]} features")
print(f"The train set has {df2.shape[0]} elements and {df2.shape[1]} features")

The train set has 16947 elements and 47 features
The train set has 2551 elements and 46 features


In [6]:
# Hypotesis: the features that contains more than 90% of NaN values 
#            do not contribute sustancial information
na_values_rate = df1.isna().sum()/len(df1)
na_values_rate = na_values_rate[na_values_rate>0]
na_values_rate

Territory                    0.294978
Source                       0.560394
Billing_Country              0.001593
Sales_Contract_No            0.411459
Account_Type                 0.006609
Brand                        0.974686
Product_Type                 0.970673
Size                         0.965422
Product_Category_B           0.970732
Price                        0.978993
Currency                     0.947188
Quote_Expiry_Date            0.272910
ASP                          0.189355
ASP_(converted)              0.189355
Planned_Delivery_End_Date    0.004426
Total_Amount                 0.003481
dtype: float64

In [7]:
# Drop the most empty features
to_drop = na_values_rate[na_values_rate > 0.90].index.to_list()

trainset = df1.drop(columns = to_drop)
testset = df2.drop(columns= to_drop)

excluded.extend(to_drop)

text = "', '".join(to_drop)
print(f"Columns '{text}' dropped")

Columns 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency' dropped


Para este modelo entonces no se consideraran las columnas ``'Brand'``, ``'Product_Type'``, ``'Size'``, ``'Product_Category_B'``, ``'Price'``, ``'Currency'``

In [8]:
# The remaining features with na_values
contains_na = na_values_rate[na_values_rate <= 0.90].index.to_list()
na_values_rate[contains_na]

Territory                    0.294978
Source                       0.560394
Billing_Country              0.001593
Sales_Contract_No            0.411459
Account_Type                 0.006609
Quote_Expiry_Date            0.272910
ASP                          0.189355
ASP_(converted)              0.189355
Planned_Delivery_End_Date    0.004426
Total_Amount                 0.003481
dtype: float64

In [9]:
def my_fill_na(df,columns):

    result = df.copy()
    dtypes = result[columns].dtypes
    cat = dtypes[dtypes == "category"].index.to_list()
    not_cat = dtypes[dtypes != "category"].index.to_list()
    
    for col in cat:
        if "Other" not in result[col].cat.categories:
            result[col].cat.add_categories("Other",inplace=True)
        result[col].fillna("Other",inplace= True)
    
    for col in not_cat:
        result[col].fillna(result[col].mean(),inplace= True)

    
    return result

In [10]:
trainset = my_fill_na(trainset,contains_na)
testset = my_fill_na(testset,contains_na)

In [11]:
na_train = trainset.isna().sum() > 0
na_test = testset.isna().sum() > 0

print(f"Restan valores nulos en trainset: {na_train.any()}")
print(f"Restan valores nulos en testset: {na_test.any()}")

Restan valores nulos en trainset: False
Restan valores nulos en testset: False


Ya filtramos las features con excesivos Nan values y rellenamos aquellas que su procentage de nan values es moderado, con valores predeterminados.

### Definición del target de los modelos

In [12]:
trainset["target"] = (trainset["Stage"] == "Closed Won").astype(int)
trainset[["Stage","target"]].head(10)

,Stage,target
ID,,
27761,Closed Lost,0
27760,Closed Won,1
27446,Closed Won,1
16808,Closed Lost,0
16805,Closed Lost,0
16802,Closed Lost,0
16799,Closed Lost,0
27455,Closed Won,1
24353,Closed Lost,0


### Definición de valores categóricos

En esta sección, lo que vamos a hacer es buscar valores categóricos de los features del set de datos de prueba que no hayan sido contemplados en el set de entrenamiento, que por lo consiguiente, el modelo de machine learning los va a desconocer.

Una vez identificados, los reemplazaremos con un valor genérico para "otros valores"

In [13]:
excluded.extend(["Stage","Opportunity_Name"])
results = []
for column in categorical_columns:
    if column in excluded: continue
    
    official_values = trainset[column].cat.categories.to_list()
    test_values = testset[column].cat.categories.to_list()
    
    other_values = set(test_values)
    for value in official_values: 
        other_values.discard(value)
    
    if len(other_values)>0:
        
        if not "Other" in testset[column].cat.categories.to_list():
            testset[column].cat.add_categories("Other",inplace=True)
        
        if not "Other" in trainset[column].cat.categories.to_list():
            trainset[column].cat.add_categories("Other",inplace=True)
            
        testset[column].replace({x:"Other" for x in other_values},inplace=True)
    
        results.append((column,len(other_values)))


In [14]:
others_df = pd.DataFrame(results,columns= ["column", "discarted values"])
others_df

,column,discarted values
0,Territory,3
1,Billing_Country,3
2,Account_Name,205
3,Opportunity_Owner,5
4,Last_Modified_By,8
5,Product_Family,20
6,Product_Name,50


In [15]:
display(round(others_df["discarted values"].describe(),2))
print(f"\nTotal test values discarted {others_df['discarted values'].sum()}, ",end="")
print(f"%{100*round(others_df['discarted values'].sum()/testset.size,3)} of total test data")

count      7.00
mean      42.00
std       73.81
min        3.00
25%        4.00
50%        8.00
75%       35.00
max      205.00
Name: discarted values, dtype: float64


Total test values discarted 294, %0.3 of total test data


La información descartada es muy poca con respecto al volumen del set. Más adelante veremos si es posible extraer información de esto

### Conversión de features

Queremos manejar los precios en una única unidad monetaria: USD

In [16]:


check1 = (trainset["Total_Amount_Currency"] == "USD").value_counts()
check2 = (trainset["Total_Taxable_Amount_Currency"] == "USD").value_counts()
check3 = (trainset["Total_Amount_Currency"]==trainset["Total_Taxable_Amount_Currency"]).value_counts()

print("Moneda de ASP == Moneda Total_Ammount :               ",check1.index[0],check1.values[0])
print("Moneda de ASP == Moneda Total_Taxable_Ammount:        ",check2.index[0],check2.values[0])
print("Moneda Total_Ammount == Moneda Total_Taxable_Ammount :",check3.index[0],check3.values[0])

Moneda de ASP == Moneda Total_Ammount :                False 9586
Moneda de ASP == Moneda Total_Taxable_Ammount:         False 9600
Moneda Total_Ammount == Moneda Total_Taxable_Ammount : True 16914


In [17]:
economy_columns = ["ASP","ASP_Currency","ASP_(converted)","ASP_(converted)_Currency","Total_Amount","Total_Amount_Currency","Total_Taxable_Amount","Total_Taxable_Amount_Currency"]
trainset.loc[:,economy_columns].sample(10)

,ASP,ASP_Currency,ASP_(converted),ASP_(converted)_Currency,Total_Amount,Total_Amount_Currency,Total_Taxable_Amount,Total_Taxable_Amount_Currency
ID,,,,,,,,
8291,46.0000,JPY,0.41356,USD,3.588000e+04,JPY,53880.00,JPY
23457,80.0000,JPY,0.71923,USD,1.993659e+06,JPY,0.00,JPY
27212,65.0000,JPY,0.58437,USD,4.777500e+05,JPY,667950.00,JPY
28524,0.5390,USD,0.53900,USD,1.621851e+06,USD,1621851.00,USD
11060,0.3600,USD,0.36000,USD,1.980000e+04,USD,82170.00,USD
12974,0.5150,USD,0.51500,USD,3.606802e+06,USD,0.00,USD
15916,0.3513,USD,0.35130,USD,1.938333e+06,USD,24544893.66,USD
22336,0.4150,USD,0.41500,USD,4.380740e+05,USD,2102755.20,USD
22818,0.5100,USD,0.51000,USD,1.963500e+05,USD,196350.00,USD


#### Calculo del factor de conversión

In [18]:
# Factor de conversión:  ASP_(converted)/ASP
trainset["USD_factor"] = trainset["ASP_(converted)"]/trainset["ASP"]
testset["USD_factor"] =  testset["ASP_(converted)"]/testset["ASP"]

print("Trainset USD_Factor describe\n")
display(trainset["USD_factor"].describe()) 

print("\n"+("-"*30))
print("\nTestset USD_Factor describe\n")
display(testset["USD_factor"].describe())

Trainset USD_Factor describe



count    1.664700e+04
mean              inf
std               NaN
min      8.865556e-03
25%      6.168651e-02
50%      1.000000e+00
75%      1.131081e+00
max               inf
Name: USD_factor, dtype: float64


------------------------------

Testset USD_Factor describe



count    2536.000000
mean        0.962582
std         0.283236
min         0.008878
25%         1.000000
50%         1.000000
75%         1.131083
max         1.131116
Name: USD_factor, dtype: float64

Tenemos valores anómalos como infinto o NaN, esto se debe a que pueden haber varios valores nulos en las columnas referidas al ASP. Lo que puede significar datos erroneos o faltantes (nunca unca conversión puede ser x/0 o 0/0 ). Procederemos a investigar estos valores:

In [19]:
# Lo que vemos que pasa es que en 300 registros, ASP y ASP_(converted) pueden valer 0 a la vez. Lo cual podría significar que no hay promedio de ventas. 
# En el unico registro que queda, ASP vale 0 y el ASP_(convertad) distinto de cero, por lo que la conversión da infinito.

print("Trainset[USD_factor] NaN values: ",trainset["USD_factor"].isna().sum())
print("Trainset[USD_factor] inf values: ",(trainset["USD_factor"]==np.inf).sum())
print("-"*36)
print("Testset[USD_factor] NaN values: ",testset["USD_factor"].isna().sum())
print("Testset[USD_factor] inf values: ",(testset["USD_factor"]==np.inf).sum())

Trainset[USD_factor] NaN values:  300
Trainset[USD_factor] inf values:  1
------------------------------------
Testset[USD_factor] NaN values:  15
Testset[USD_factor] inf values:  0


In [20]:
economy_features= ["Opportunity_ID","ASP","ASP_Currency","ASP_(converted)","USD_factor","Total_Amount",
                   "Total_Amount_Currency","Total_Taxable_Amount","Total_Taxable_Amount_Currency"]

condition = (trainset["USD_factor"]==np.inf)| ((trainset["ASP"]==0) & (trainset["ASP_(converted)"]==0))
selected = trainset.loc[condition,economy_features]
selected

,Opportunity_ID,ASP,ASP_Currency,ASP_(converted),USD_factor,Total_Amount,Total_Amount_Currency,Total_Taxable_Amount,Total_Taxable_Amount_Currency
ID,,,,,,,,,
24319,41,0.0,JPY,0.0,NaN,20280.0,JPY,20280.0,JPY
19082,105,0.0,JPY,0.0,NaN,34000.0,JPY,390050.0,JPY
19085,105,0.0,JPY,0.0,NaN,26000.0,JPY,390050.0,JPY
19089,105,0.0,JPY,0.0,NaN,1200.0,JPY,390050.0,JPY
19092,105,0.0,JPY,0.0,NaN,6200.0,JPY,390050.0,JPY
...,...,...,...,...,...,...,...,...,...
19011,12514,0.0,EUR,0.0,NaN,0.0,EUR,0.0,EUR
28261,12520,0.0,EUR,0.0,NaN,0.0,EUR,0.0,EUR
19923,12659,0.0,EUR,0.0,NaN,0.0,EUR,0.0,EUR


In [21]:
condition = (trainset["USD_factor"]==np.inf) | (trainset["USD_factor"].isna()) 
print("(Trainset) Registros 'USD_factor == Nan' con currency en USD: ",(condition&(trainset["ASP_Currency"]=="USD")).sum())

condition = (testset["USD_factor"]==np.inf) | (testset["USD_factor"].isna()) 
print("(Testset) Registros 'USD_factor == Nan' con currency en USD: ",(condition&(testset["ASP_Currency"]=="USD")).sum())

(Trainset) Registros 'USD_factor == Nan' con currency en USD:  54
(Testset) Registros 'USD_factor == Nan' con currency en USD:  2


In [22]:
# Veo casos particulares donde la conversion debería ser 1 y no lo es
len1 = trainset[(trainset["ASP_Currency"]=="USD")&(trainset["USD_factor"] != 1)][economy_features+["ASP_(converted)_Currency"]].shape[0]
len2 = testset[(testset["ASP_Currency"]=="USD")&(testset["USD_factor"] != 1)][economy_features+["ASP_(converted)_Currency"]].shape[0]

print("Registros donde se maneja USD en todas sus columnas económicas y la conversión no es 1")
print(f"Trainset:  {len1}\t ({round(100*len1/trainset.shape[0],1)}% del set)")
print(f"Testset:   {len2}\t ({round(100*len2/testset.shape[0],1)}% del set)")

Registros donde se maneja USD en todas sus columnas económicas y la conversión no es 1
Trainset:  525	 (3.1% del set)
Testset:   64	 (2.5% del set)


In [23]:
# Nos aseguramos que los casos en donde la moneda sea dolares, la conversión sea 1

trainset.loc[trainset["ASP_Currency"]=="USD","USD_factor"] = 1
testset.loc[testset["ASP_Currency"]=="USD","USD_factor"] = 1

In [24]:
originalTrain = trainset.loc[(trainset["Total_Amount_Currency"] == "USD")|
                             (trainset["Total_Taxable_Amount_Currency"] == "USD"),["Total_Amount","Total_Taxable_Amount"]]
originalTest = testset.loc[(testset["Total_Amount_Currency"] == "USD")|
                             (testset["Total_Taxable_Amount_Currency"] == "USD"),["Total_Amount","Total_Taxable_Amount"]]

Finalmente, terminamos de calcular/estimar el factor de conversión y convertimos los precios

In [25]:
# Vamos a realzar un Mean Encoding para calcular el factor de conversión de la moneda como un promedio histórico
# Para finalmente realizar la conversión

################ Calculating Mean Values ################
# Train
trainset["ASP_Mean"]             = trainset.groupby("ASP_Currency")["ASP"].transform("mean")
trainset["ASP_Mean_(converted)"] = trainset.groupby("ASP_Currency")["ASP_(converted)"].transform("mean")

# Test
testset["ASP_Mean"]              = testset.groupby("ASP_Currency")["ASP"].transform("mean")
testset["ASP_Mean_(converted)"]  = testset.groupby("ASP_Currency")["ASP_(converted)"].transform("mean")


########### Mean encoding for the NAN values #############
# Train
condition = (trainset["USD_factor"].isna()) | (trainset["USD_factor"] == np.inf)
trainset.loc[condition,"USD_factor"] = trainset[condition]["ASP_Mean_(converted)"]/\
                                       trainset[condition]["ASP_Mean"]

# Test
condition = (testset["USD_factor"].isna()) | (testset["USD_factor"] == np.inf)
testset.loc[condition,"USD_factor"] = testset[condition]["ASP_Mean_(converted)"]/\
                                      testset[condition]["ASP_Mean"]

In [26]:
##################### Conversion #####################

# ------------------Train------------------------

# Initcializo y luego convierto
trainset["Total_Amount_USD"]         = trainset["Total_Amount"]
trainset["Total_Taxable_Amount_USD"] = trainset["Total_Taxable_Amount"]

# Conversión de Total_Amount
condition = trainset["Total_Amount_Currency"] != "USD" 
trainset.loc[condition,"Total_Amount_USD"]     = trainset[condition]["Total_Amount"]*\
                                                 trainset[condition]["USD_factor"]

# Conversión de Total_Taxable_Amount
condition = trainset["Total_Taxable_Amount_Currency"] != "USD" 
trainset.loc[condition,"Total_Taxable_Amount_USD"] = trainset[condition]["Total_Taxable_Amount"]*\
                                                     trainset[condition]["USD_factor"]

# ------------------Test------------------------

# Initcializo y luego convierto
testset["Total_Amount_USD"]         = testset["Total_Amount"]
testset["Total_Taxable_Amount_USD"] = testset["Total_Taxable_Amount"]

# Conversión de Total_Amount
condition = testset["Total_Amount_Currency"] != "USD" 
testset.loc[condition,"Total_Amount_USD"] = testset[condition]["Total_Amount"]*\
                                            testset[condition]["USD_factor"]
# Conversión de Total_Taxable_Amount
condition = testset["Total_Taxable_Amount_Currency"] != "USD"
testset.loc[condition,"Total_Taxable_Amount_USD"] = testset[condition]["Total_Taxable_Amount"]*\
                                                    testset[condition]["USD_factor"]

excluded.extend(["Total_Amount","Total_Taxable_Amount"])

# Feature engineering

Objetivos:

   * 'Aplanar' varios registros de una misma oportunidad, en un solo registro
   * Encodear variables categóricas a partir de información de otros features
   * Encodear variables categóricas con binary encoding
   * Crear interacciones nuevas entre features

### Flatten opportunities

En el análisis exploratorio, revelamos que las oportunidades pueden tener un solo item o varios, dejando varios registros en el set de datos. Tambien, vimos cuales son las features que pueden variar en los items de una misma oportunidad.

En base a esto, vamos a hacer feature engineering para poder extraer información de estos distintos valores que pueden tomar estas features, y dejarla plasmada en una sola feature que califique a la oportunidad en cuestión.

In [27]:
# Evaluamos cada feature que registra variaciones para una misma oportunidad
# y proponemos una manera de "aplanar" esa variacion en un solo valor.

#  "Product_Name"               # Hacer bins segun mas vendido y contabilizar apariciones en los bins | promedio de aparicion y promediar los items 
#  "Product_Family"             # Hacer bins segun mas vendido y contabilizar apariciones en los bins | promedio de aparicion y promediar los items
#  "Total_Amount"               # Promedios, desvio estandar
#  "Planned_Delivery_End_Date"  # Rango maximo, Rango minimo, Rango promedio  \___
#  "Planned_Delivery_Start_Date"# Rango maximo, Rango minimo, Rango promedio  /
#  "ASP_(converted)"            # Promedios, desvio estandar
#  "ASP"                        # [X] descartado, pordría indicarse con la anterior
#  "TRF"                        # Promedios, desvio estandar
#  "Delivery_Quarter"           # [X] si varía podría ser porque cambien los planned_delivery dates..
#  "Delivery_Year"              # [X] ..
#  "Month"                      # [X] .. y casualmente cambie de año o de trimestre. Descartamos esta variación
                        

to_flat=   ["Product_Name",
           "Product_Family",
           "Total_Amount",
           "Planned_Delivery_End_Date",
           "Planned_Delivery_Start_Date",
           "ASP_(converted)",
           "ASP",
           "TRF",
           "Delivery_Quarter",     
           "Delivery_Year",
           "Month"]

In [28]:
grouped = trainset.groupby(["Product_Family","Product_Name"]).agg({"target":"count"})
grouped = grouped.reset_index()
grouped    

,Product_Family,Product_Name,target
0,Product_Family_0,Product_Name_0,165
1,Product_Family_0,Product_Name_1,0
2,Product_Family_0,Product_Name_10,0
3,Product_Family_0,Product_Name_100,0
4,Product_Family_0,Product_Name_101,0
...,...,...,...
104191,Other,Product_Name_504,0
104192,Other,Product_Name_510,0
104193,Other,Product_Name_512,0
104194,Other,Product_Name_87,0


In [29]:
print(f"Cantidad de combinaciones familia-producto con por lo menos 1 registro: {len(grouped.loc[grouped['target']>0])}")
print(f"Cantidad de valores de Product_Name: {len(grouped['Product_Name'].drop_duplicates())}")

Cantidad de combinaciones familia-producto con por lo menos 1 registro: 456
Cantidad de valores de Product_Name: 457


Esto nos confirma que cada producto pertenece a una sola familia excepto "Otros" que no pertenece a ninguna. Para realizar el "Flatten" de productos elegidos en una oportunidad, usamos su familia.

In [30]:
grouped = trainset["Product_Family"].value_counts()
families = { k:v for k,v in grouped.items()}

grouped = trainset["Product_Name"].value_counts()
products = { k:v for k,v in grouped.items()}

In [31]:
# Flatten opportunity_ID

# 
trainset["Family_Group"] = trainset["Product_Family"].agg(lambda x: families[x])
trainset["Product_Group"] = trainset["Product_Name"].agg(lambda x: products[x])
trainset["Planned_range"] =  trainset["Planned_Delivery_End_Date"] - trainset["Planned_Delivery_Start_Date"]

testset["Family_Group"] = testset["Product_Family"].agg(lambda x: families[x])
testset["Product_Group"] = testset["Product_Name"].agg(lambda x: products[x])
testset["Planned_range"] =  testset["Planned_Delivery_End_Date"] - testset["Planned_Delivery_Start_Date"]


In [32]:
trainset["Family_group_mean"]   = trainset.groupby("Opportunity_ID")["Family_Group"].transform("mean")
trainset["Product_group_mean"]  = trainset.groupby("Opportunity_ID")["Product_Group"].transform("mean")
trainset["Items"]               = trainset.groupby("Opportunity_ID")["target"].transform("count")
trainset["Total_Amount_mean"]   = trainset.groupby("Opportunity_ID")["Total_Amount_USD"].transform("mean")  #Promedio dolarizado
trainset["ASP_(converted)_mean"]= trainset.groupby("Opportunity_ID")["ASP_(converted)"].transform("mean")
trainset["TRF_min"]             = trainset.groupby("Opportunity_ID")["TRF"].transform("min")
trainset["TRF_max"]             = trainset.groupby("Opportunity_ID")["TRF"].transform("max")
trainset["Planned_min_range"]   = trainset.groupby("Opportunity_ID")["Planned_range"].transform("min")
trainset["Planned_max_range"]   = trainset.groupby("Opportunity_ID")["Planned_range"].transform("max")
trainset["Planned_min_range"]   = trainset["Planned_min_range"].dt.days
trainset["Planned_max_range"]   = trainset["Planned_max_range"].dt.days
trainset.info()

testset["Family_group_mean"]    = testset.groupby("Opportunity_ID")["Family_Group"].transform("mean")
testset["Product_group_mean"]   = testset.groupby("Opportunity_ID")["Product_Group"].transform("mean")
testset["Items"]                = testset.groupby("Opportunity_ID")["Opportunity_Name"].transform("count")
testset["Total_Amount_mean"]    = testset.groupby("Opportunity_ID")["Total_Amount_USD"].transform("mean")
testset["ASP_(converted)_mean"] = testset.groupby("Opportunity_ID")["ASP_(converted)"].transform("mean")
testset["TRF_min"]              = testset.groupby("Opportunity_ID")["TRF"].transform("min")
testset["TRF_max"]              = testset.groupby("Opportunity_ID")["TRF"].transform("max")
testset["Planned_min_range"]    = testset.groupby("Opportunity_ID")["Planned_range"].transform("min")
testset["Planned_max_range"]    = testset.groupby("Opportunity_ID")["Planned_range"].transform("max")
testset["Planned_min_range"]    = testset["Planned_min_range"].dt.days
testset["Planned_max_range"]    = testset["Planned_max_range"].dt.days
testset.info()

excluded.extend(to_flat)
excluded.extend(["Family_Group","Product_Group","Planned_range"])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16947 entries, 27761 to 28318
Data columns (total 59 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Region                              16947 non-null  category       
 1   Territory                           16947 non-null  category       
 2   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64          
 3   Pricing, Delivery_Terms_Approved    16947 non-null  int64          
 4   Bureaucratic_Code_0_Approval        16947 non-null  int64          
 5   Bureaucratic_Code_0_Approved        16947 non-null  int64          
 6   Bureaucratic_Code                   16947 non-null  category       
 7   Account_Created_Date                16947 non-null  datetime64[ns] 
 8   Source                              16947 non-null  category       
 9   Billing_Country                     16947 non-null  category       
 10  Accoun

### Mean encoding

Proponemos encodear la columna "Territorio" como el promedio de los TRF solicitados en esa región. Nos basamos en el negocio mismo, el cual ofrece equipos de refrigeración, que se suponen que serán más solicitados en cantidad y en potencia (TRF) en regiones donde hace más calor. Es por esto que creemos que el promedio de los TRF, en un territorio dado, califica conceptualmente bien a ese territorio.



In [33]:
import math

n= len(trainset["Territory"].value_counts().index.to_list())
print("Columnas de Binary-Encoding ahorradas: ",int(math.log2(n))+1)
print("Columnas usadas con mean encoding: 2")

Columnas de Binary-Encoding ahorradas:  7
Columnas usadas con mean encoding: 2


In [34]:
trainset["Territory_TRF_mean"]  = trainset.groupby("Territory")["TRF"].transform("mean")
trainset["Territory_TRF_count"]  = trainset.groupby("Territory")["TRF"].transform("count")
trainset["Territory_TRF_max"]  = trainset.groupby("Territory")["TRF"].transform("max")

testset["Territory_TRF_mean"]  = testset.groupby("Territory")["TRF"].transform("mean")
testset["Territory_TRF_count"]  = testset.groupby("Territory")["TRF"].transform("count")
testset["Territory_TRF_max"]  = testset.groupby("Territory")["TRF"].transform("max")

excluded.extend(["Territory"])

In [35]:
testset[["Territory","TRF"]].head(5)

,Territory,TRF
ID,,
6140,Germany,0
6146,Germany,0
6151,Germany,0
6118,Germany,1
6124,Germany,0


### New features from others

Crearemos columnas referidas al intervalo en dias que hay entre dos fechas relevalntes

In [36]:
date_columns

['Account_Created_Date',
 'Opportunity_Created_Date',
 'Quote_Expiry_Date',
 'Last_Modified_Date',
 'Planned_Delivery_Start_Date',
 'Planned_Delivery_End_Date']

* `Opportunity_Created_Date` y `Quote_Expiry_Date`


In [37]:
trainset["QuoteExpiry-OppCreated"] = (trainset["Quote_Expiry_Date"]-trainset["Opportunity_Created_Date"]).dt.days
testset["QuoteExpiry-OppCreated"]  = (testset["Quote_Expiry_Date"] -testset["Opportunity_Created_Date"]).dt.days

* `Opportunity_Created_Date` y `Account_Created_Date`

In [38]:
trainset["OppCreated-AccCreated"] = (trainset["Opportunity_Created_Date"]-trainset["Account_Created_Date"]).dt.days
testset["OppCreated-AccCreated"]  = (testset["Opportunity_Created_Date"] -testset["Account_Created_Date"]).dt.days

* `Opportunity_Created_Date` y `Last_Modified_Date`

In [39]:
trainset["OppCreated-LastMod"] = (trainset["Opportunity_Created_Date"]-trainset["Last_Modified_Date"]).dt.days
testset["OppCreated-LastMod"]  = (testset["Opportunity_Created_Date"] -testset["Last_Modified_Date"]).dt.days

* `Opportunity_Created_Date` y `Planned_Delivery_Start_Date`


In [40]:
trainset["PlanStart-OppCreated"] = (trainset["Planned_Delivery_Start_Date"]-trainset["Opportunity_Created_Date"]).dt.days
testset["PlanStart-OppCreated"]  = (testset["Planned_Delivery_Start_Date"] -testset["Opportunity_Created_Date"]).dt.days

### One-hot encoding 
Realizaremos un dataset básico para poder correr el modelo por primera vez y observar los resultados.
Luego realizaremos mejoras e ingeniería de features para ver como se comporta el modelo.

In [41]:
from sklearn.preprocessing import OneHotEncoder 

In [42]:
print(f"El set de entrenamiento tiene {trainset.size} elementos")
print(f"El set de test tiene {testset.size} elementos")

El set de entrenamiento tiene 1118502 elementos
El set de test tiene 163264 elementos


In [43]:
excluded.extend(["ASP_(converted)_Currency","Total_Taxable_Amount_Currency",'target'])

toEncode = set([col if col not in excluded else "" for col in categorical_columns])
toEncode.discard("")
toEncode = list(toEncode)

print("Features categóricos a encodear")
toEncode

Features categóricos a encodear


['Account_Name',
 'Account_Owner',
 'Delivery_Terms',
 'Last_Modified_By',
 'Bureaucratic_Code',
 'ASP_Currency',
 'Billing_Country',
 'Quote_Type',
 'Total_Amount_Currency',
 'Region',
 'Opportunity_Type',
 'Opportunity_Owner',
 'Source',
 'Account_Type']

In [44]:
enc = OneHotEncoder(drop='if_binary')
enc.fit(trainset[toEncode])
ohed = pd.DataFrame(enc.transform(trainset[toEncode]).toarray())
print(f"Shape: {ohed.shape}")
print(f"Size: {ohed.size}")
ohed.info()

Shape: (16947, 1949)
Size: 33029703
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Columns: 1949 entries, 0 to 1948
dtypes: float64(1949)
memory usage: 252.0 MB


Decidimos abandonar esta propuesta debido a la poca escalabilidad que tiene este método. Debido a las numerosas columnas categóricas con numerosos valores posibles cada una, necesitamos un total de ``2.723`` columnas para `16.947` entradas, lo cual nos deja un total de `46.146.681` valores en nuestra tabla.

### Binary encoding 

Este método nos permite encodear las features categóricas reduciendo considerablemente el dataset. Si suponemos que los 16.947 valores categóricos posibles se distribuyen uniformemente en las 7 columnas, entonces necesitaremos ``⌊log_2(16.947/7)⌋ + 1  = 9`` columnas nuevas por cada columna categórica original del dataset. En total ``9*7 = 63`` una propuesta cuestionable pero mucho mejor que la anterior. 

In [45]:
import math
class BinaryEncoding():
    
    def __init__(self):
        self.__encodings = {}
        
    def __make_encoding(self,name,categories):

        encoding = {}
        n_cols = int(math.log(len(categories),2))+1
        cols_names = [ name+"_"+str(x) for x in range(n_cols) ]

        for i in range(len(categories)):
            encoding[categories[i]] = list(f'{bin(i)[2:]}'.zfill(n_cols))

        return (encoding,cols_names)
    
    def encode_Series(self, serie, name,verbose=False):
        
        
        if not name in self.__encodings:

            categories = serie.cat.categories.to_list()
            self.__encodings[name] =  self.__make_encoding(name,categories)
        
        encoding,cols_names = self.__encodings[name]
        data = []
        indexs = []
        
        for index, value in serie.items():

            data.append(encoding[value])
            indexs.append(index)

        df_result = pd.DataFrame(data,columns=cols_names,index=indexs)
        return df_result,cols_names
    
    def getEncoding(self):
        return self.__encodings
    
    def encode_DataFrame(self, df, toEncode, verbose = False):

        full_encoded,columns = self.encode_Series(df[toEncode[0]],toEncode[0],verbose)
       
        for col in toEncode[1:]:
            
            encoding,col_names = self.encode_Series(df[col],col,verbose)
            full_encoded[col_names] = encoding

        if verbose: self.evaluate_encoding(df[toEncode],full_encoded)

        return full_encoded
    
    def evaluate_encoding(self,original,encoded):

        # Recuento de cada una de las combinaciones de 
        # la lista de features categóricos sin encodear
        count1 = original.value_counts().to_frame()[0].values

        # Recuento de cada una de las combinaciones de 
        # la lista de features categóricos ENCODEADOS
        count2 = encoded.value_counts().to_frame()[0].values

        # Comparación
        print("El encoding fue realizado correctamente: ", np.equal(count1,count2).all())


In [46]:
encoder = BinaryEncoding()
trainEncoded = encoder.encode_DataFrame(trainset,toEncode,verbose=True)
testEncoded = encoder.encode_DataFrame(testset,toEncode,verbose=True)

El encoding fue realizado correctamente:  True
El encoding fue realizado correctamente:  True


In [47]:
print(f"Train-encoded shape: {trainEncoded.shape}")
print(f"Train-encoded size: {trainEncoded.size}")
print("--------------------------------")
print(f"Test-encoded shape: {testEncoded.shape}")
print(f"Test-encoded size: {testEncoded.size}")

Train-encoded shape: (16947, 66)
Train-encoded size: 1118502
--------------------------------
Test-encoded shape: (2551, 66)
Test-encoded size: 168366


Pudimos encodear las columnas categóricas reduciendo notablemente el espacio. El cálculo de columnas utilizadas previo valía con la suposición de que todas las features tenían la misma cantidad de valores. Pero aún no cumpliendose, la diferencia de columnas es poca con respecto al resultado obtenido con One Hot Encoding

Terminamos este procesamiento de datos, para ver como un modelo de RandomForest se comporta frente a esto.

In [48]:
print("Excluidas:\n")
display(excluded)

print("\nEncodeadas:\n")
display(toEncode)

Excluidas:



['Actual_Delivery_Date',
 'Last_Activity',
 'Submitted_for_Approval',
 'Prod_Category_A',
 'Brand',
 'Product_Type',
 'Size',
 'Product_Category_B',
 'Price',
 'Currency',
 'Stage',
 'Opportunity_Name',
 'Total_Amount',
 'Total_Taxable_Amount',
 'Product_Name',
 'Product_Family',
 'Total_Amount',
 'Planned_Delivery_End_Date',
 'Planned_Delivery_Start_Date',
 'ASP_(converted)',
 'ASP',
 'TRF',
 'Delivery_Quarter',
 'Delivery_Year',
 'Month',
 'Family_Group',
 'Product_Group',
 'Planned_range',
 'Territory',
 'ASP_(converted)_Currency',
 'Total_Taxable_Amount_Currency',
 'target']


Encodeadas:



['Account_Name',
 'Account_Owner',
 'Delivery_Terms',
 'Last_Modified_By',
 'Bureaucratic_Code',
 'ASP_Currency',
 'Billing_Country',
 'Quote_Type',
 'Total_Amount_Currency',
 'Region',
 'Opportunity_Type',
 'Opportunity_Owner',
 'Source',
 'Account_Type']

In [49]:
excluded.extend(date_columns+['ASP_Mean','ASP_Mean_(converted)'])
final_cols = list()
categorical_columns.append("target")
for col in trainset.columns.to_list():
    if not col in excluded+categorical_columns:
        final_cols.append(col)
print(f"Selected {len(final_cols)} features no categóricos")
final_cols

Selected 25 features no categóricos


['Pricing, Delivery_Terms_Quote_Appr',
 'Pricing, Delivery_Terms_Approved',
 'Bureaucratic_Code_0_Approval',
 'Bureaucratic_Code_0_Approved',
 'Opportunity_ID',
 'Sales_Contract_No',
 'USD_factor',
 'Total_Amount_USD',
 'Total_Taxable_Amount_USD',
 'Family_group_mean',
 'Product_group_mean',
 'Items',
 'Total_Amount_mean',
 'ASP_(converted)_mean',
 'TRF_min',
 'TRF_max',
 'Planned_min_range',
 'Planned_max_range',
 'Territory_TRF_mean',
 'Territory_TRF_count',
 'Territory_TRF_max',
 'QuoteExpiry-OppCreated',
 'OppCreated-AccCreated',
 'OppCreated-LastMod',
 'PlanStart-OppCreated']

In [50]:
finalTrain = trainset.loc[:,final_cols+["target"]]
finalTrain[trainEncoded.columns.to_list()] = trainEncoded
finalTrain.drop_duplicates("Opportunity_ID",inplace=True)
finalTrain.shape

(9841, 92)

In [51]:
finalTest = testset.loc[:,final_cols]
finalTest[testEncoded.columns.to_list()] = testEncoded
finalTest.drop_duplicates("Opportunity_ID",inplace=True)
finalTest.shape

(1567, 91)

In [52]:
finalTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9841 entries, 27761 to 28318
Data columns (total 92 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Pricing, Delivery_Terms_Quote_Appr  9841 non-null   int64  
 1   Pricing, Delivery_Terms_Approved    9841 non-null   int64  
 2   Bureaucratic_Code_0_Approval        9841 non-null   int64  
 3   Bureaucratic_Code_0_Approved        9841 non-null   int64  
 4   Opportunity_ID                      9841 non-null   int64  
 5   Sales_Contract_No                   9841 non-null   float64
 6   USD_factor                          9841 non-null   float64
 7   Total_Amount_USD                    9841 non-null   float64
 8   Total_Taxable_Amount_USD            9841 non-null   float64
 9   Family_group_mean                   9841 non-null   float64
 10  Product_group_mean                  9841 non-null   float64
 11  Items                               98

In [53]:
# Se eliminaron las columnas vacías o con valores iguales
# Se eliminaron features con mas del 90% de registros NaN
# Se rellenaron nan_values con promedios para features numéricos y con "Others" para categóricos
# Se eliminaron valores categóricos del set de test que no estan en el set de entrenamiento
# Se convirtieron valores monetarios en distintas monedas a valores dolarizados
# Se unificaron los items en un mismo regirstro con igual Opportunity_ID.
# Se realizo Mean Encoding para el territory segun su TRF
# Se realizó binary encoding para todos las features categóricas
# Se crearon features relacionadas a rangos en días entre dos fechas
# Se agregó la columna "target" la cual tiene 1/0 según es "Closed Won" o no
# Se eliminó la columna "Stage"


# feature engineering

path = "datasets/"
name = "third-rf-model(2)"
finalTrain.to_csv(path+name+"-train.csv", index = False)
finalTest.to_csv(path+name+"-test.csv", index = False)